__Inference set up - Basemodel_FMD__

In [ ]:
from fastai.vision.all import *
import os
import pathlib
import fastai; fastai.__version__

In [ ]:
torch.cuda.set_device(0)

In [ ]:
torch.cuda.current_device(), torch.cuda.get_device_name(0)

In [ ]:
device=torch.device("cuda")

In [ ]:
path = pathlib.Path(os.getcwd())

In [ ]:
path_GT = path/'Daten/FMD_GT'
path_noisy = path/'Daten/FMD_noisy'

In [ ]:
set_seed(42)

In [ ]:
%run utility/Dataloader_FMD.ipynb
%run utility/Perception_loss.ipynb

In [ ]:
dls_den = get_dls(38, 256)  # required to generate learner       

In [ ]:
loss_func = FeatureLoss(vgg_m, blocks[2:5],[1], [1,1,1], [1,1,1]) # dummy

In [ ]:
# define learner w/o learning parameters (e.g. metrics, cbs)
def create_gen_learner():
    return unet_learner(dls_den, bbone, loss_func=loss_func, blur=True, norm_type=NormType.Weight, 
                        self_attention=True, y_range=y_range
                        )

In [ ]:
# create learner
learn_den = create_gen_learner()
# load the Basemodel into that leaner and convert to fp16
learn_den.load('Basemodel_FMD').to_fp16

__Set up data and perform inference__

In [ ]:
path_test = path/'Daten/FMD_testmix/noisy_tiles'
path_preds = path/'Daten/FMD_testmix/FMD_pred_tiles'
path_preds.mkdir(exist_ok=True)

In [ ]:
# check path
test_files = get_image_files(path_test)
test_files[0], len(test_files)

In [ ]:
# create test set dataloader
test_dl = learn_den.dls.test_dl(test_files)

In [ ]:
# generate predictions, decode oop over all predictions and save them as 8 bit png
def save_preds(dl, learn):
  "Save away predictions"
  names = dl.dataset.items
  
  inp, preds, _,decoded = learn.get_preds(dl=dl, with_input=True, with_decoded=True)
  for i,pred in enumerate(preds):
      dec = dl.after_batch.decode((TensorImage(pred[None]),))[0][0]
      arr = dec.numpy().transpose(1,2,0).astype(np.uint8)
      Image.fromarray(arr[:,:,0]).save(path_preds/names[i].name)
  return inp, preds, decoded       # return outputs for analysis

In [ ]:
inp, preds, dec = save_preds(test_dl, learn_den)

In [ ]:
# use this loop with a chunksize that works for your RAM, if your RAM cant contain all predictions simultaneously
"""
chunk_size = 5000
for k in range(0,len(test_files),chunk_size):
    chunk = test_files[k:k+chunk_size]
    test_dl=learn_den.dls.test_dl(chunk)
    inp, preds, dec = save_preds(test_dl, learn_den)
"""